# Learning From Data : First Coding Assessment

## data
- source: https://www.kaggle.com/datasets/willianoliveiragibin/bank-churn-prediction


## about
In the synthetic dataset for the Playground Series S4 E1 Binary Classification with a Bank Churn Dataset, various features have been engineered to capture relevant information about customers. 
- The dataset includes label-encoded surnames and features derived from them using the TFIDF vectorizer. 
- The credit score serves as a numerical representation of a customer's creditworthiness.
- Geography feature indicates the country of residence, with one-hot encoding for France, Spain, and Germany.
- Gender is represented with one-hot encoding for male and female categories.
- Age, tenure, balance, and the number of products used by the customer offer insights into their banking behavior.
- The presence of a credit card, active membership status, and estimated salary are also included as binary features.
- Notable engineered features provide additional insights. 
- Mem__no__Products is the product of the number of products and active membership status, offering a combined metric. 
- Cred_Bal_Sal represents the ratio of the product of credit score and balance to estimated salary, providing a relative measure of financial health. 
- The balance-to-salary ratio (Bal_sal) and the tenure-to-age ratio (Tenure_Age) offer further dimensions for analysis. 
- Finally, Age_Tenure_product is a feature capturing the interaction between age and tenure.
- The target variable, 'Exited,' indicates whether a customer has churned, with a value of 1 for churned customers and 0 for those who have not. 

This dataset, with its diverse set of features and engineered metrics, provides a comprehensive foundation for binary classification tasks, enabling the exploration of factors influencing customer churn in the banking domain. 
Analysts and data scientists can leverage these features to build predictive models and gain insights into the dynamics of customer retention.

In [37]:
import pandas as pd
import numpy as np
import random

random.seed(18)
np.random.seed(18)
pd.set_option('display.max_columns', 25)

In [38]:
df = pd.read_csv("./data/bank_churn_small.csv")
df.head()

,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Surname_tfidf_0,Surname_tfidf_1,Surname_tfidf_2,Surname_tfidf_3,Surname_tfidf_4,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,2023,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,"-3.124.436.679,76","-460.594.640,32","13.351.443.755,82","-6.116.396.019,04","6.694.577.725,07",1,0,0,0,1,0.0,0.0,0.0,"9.090.909.090.909.090,00",99.0
1,2024,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,"481.338.256,69","838.931.547,32","-11.828.148.560,09","7.812.514.313,22","1.663.329.080,86",1,0,0,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0
2,1236,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,"-3.447.294.172.650.730,00","-2.025.033.659.055.650,00","-10.280.719.705.962.700,00","-938.715.514.665.035,00","15.068.643.143.363.400,00",1,0,0,0,1,0.0,0.0,0.0,0.25,400.0
3,1362,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,"-72.209.360.028,78","-10.461.631.817.033.800,00","131.291.330.459.128,00","846.300.277.850.006,00","16.075.843.065.996.200,00",1,0,0,0,1,1.0,"10.229.405.812.711.500,00","17.606.550.452.171.200,00","58.823.529.411.764.700,00",68.0
4,491,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,"-1.037.134.935,85","-545.133.813,64","498.455.946,81","-1.568.877.112,96","18.489.187.336,37",0,0,1,0,1,2.0,0.0,0.0,"15.151.515.151.515.100,00",165.0


# Data Loading

In [39]:
# df.tail()
df

,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Surname_tfidf_0,Surname_tfidf_1,Surname_tfidf_2,Surname_tfidf_3,Surname_tfidf_4,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,2023,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,"-3.124.436.679,76","-460.594.640,32","13.351.443.755,82","-6.116.396.019,04","6.694.577.725,07",1,0,0,0,1,0.0,0.0,0.0,"9.090.909.090.909.090,00",99.0
1,2024,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,"481.338.256,69","838.931.547,32","-11.828.148.560,09","7.812.514.313,22","1.663.329.080,86",1,0,0,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0
2,1236,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,"-3.447.294.172.650.730,00","-2.025.033.659.055.650,00","-10.280.719.705.962.700,00","-938.715.514.665.035,00","15.068.643.143.363.400,00",1,0,0,0,1,0.0,0.0,0.0,0.25,400.0
3,1362,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,"-72.209.360.028,78","-10.461.631.817.033.800,00","131.291.330.459.128,00","846.300.277.850.006,00","16.075.843.065.996.200,00",1,0,0,0,1,1.0,"10.229.405.812.711.500,00","17.606.550.452.171.200,00","58.823.529.411.764.700,00",68.0
4,491,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,"-1.037.134.935,85","-545.133.813,64","498.455.946,81","-1.568.877.112,96","18.489.187.336,37",0,0,1,0,1,2.0,0.0,0.0,"15.151.515.151.515.100,00",165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,165,678.0,33.0,7.0,129735.30,1.0,1,0,86925.09,0,"32.773.960,60","135.956.201.186,88","124.862.989.792,27","-119.249.709.468,16","-27.162.696.939,08",1,0,0,1,0,0.0,"10.119.119.048.366.800,00","14.924.954.348.623.600,00","21.212.121.212.121.200,00",231.0
3998,567,692.0,26.0,2.0,0.00,1.0,1,1,47729.33,0,"-14.010.026.435,54","240.786.132.169,47","-148.134.008.874,58","69.077.948.529,11","217.534.326.224,86",1,0,0,0,1,1.0,0.0,0.0,"7.692.307.692.307.690,00",52.0
3999,490,699.0,41.0,2.0,134870.79,1.0,1,1,105961.48,0,"150.700,02","10.921.328,16","-31.563.936,85","6.759.335,61","-9.709.299,32",1,0,0,1,0,1.0,"8.897.071.106.405.830,00","12.728.284.844.643.500,00","4.878.048.780.487.800,00",82.0
4000,2089,727.0,33.0,1.0,0.00,2.0,1,1,56767.67,0,"17.489.429.154.610.700,00","10.703.758.221.371.100,00","-38.816.038.033.103.800,00","5.973.640.018.037.230,00","8.703.513.165.834.530,00",0,0,1,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0


# Data Cleaning

In [40]:
df.head()

,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Surname_tfidf_0,Surname_tfidf_1,Surname_tfidf_2,Surname_tfidf_3,Surname_tfidf_4,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,2023,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,"-3.124.436.679,76","-460.594.640,32","13.351.443.755,82","-6.116.396.019,04","6.694.577.725,07",1,0,0,0,1,0.0,0.0,0.0,"9.090.909.090.909.090,00",99.0
1,2024,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,"481.338.256,69","838.931.547,32","-11.828.148.560,09","7.812.514.313,22","1.663.329.080,86",1,0,0,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0
2,1236,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,"-3.447.294.172.650.730,00","-2.025.033.659.055.650,00","-10.280.719.705.962.700,00","-938.715.514.665.035,00","15.068.643.143.363.400,00",1,0,0,0,1,0.0,0.0,0.0,0.25,400.0
3,1362,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,"-72.209.360.028,78","-10.461.631.817.033.800,00","131.291.330.459.128,00","846.300.277.850.006,00","16.075.843.065.996.200,00",1,0,0,0,1,1.0,"10.229.405.812.711.500,00","17.606.550.452.171.200,00","58.823.529.411.764.700,00",68.0
4,491,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,"-1.037.134.935,85","-545.133.813,64","498.455.946,81","-1.568.877.112,96","18.489.187.336,37",0,0,1,0,1,2.0,0.0,0.0,"15.151.515.151.515.100,00",165.0


In [41]:
df.columns

Index(['Surname', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_2',
       'Surname_tfidf_3', 'Surname_tfidf_4', 'France', 'Germany', 'Spain',
       'Female', 'Male', 'Mem__no__Products', 'Cred_Bal_Sal', 'Bal_sal',
       'Tenure_Age', 'Age_Tenure_product'],
      dtype='object')

### Drop columns that are not useful for data visualisation | analysis surnames

In [42]:
df.drop(
  columns=['Surname', 'Surname_tfidf_0', 'Surname_tfidf_1', 'Surname_tfidf_2', 'Surname_tfidf_3', 'Surname_tfidf_4'], inplace= True, axis=1
)
print(df.columns.shape)
df.head()

(19,)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product
0,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,1,0,0,0,1,0.0,0.0,0.0,"9.090.909.090.909.090,00",99.0
1,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,1,0,0,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0
2,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,1,0,0,0,1,0.0,0.0,0.0,0.25,400.0
3,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,1,0,0,0,1,1.0,"10.229.405.812.711.500,00","17.606.550.452.171.200,00","58.823.529.411.764.700,00",68.0
4,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,0,0,1,0,1,2.0,0.0,0.0,"15.151.515.151.515.100,00",165.0


### Put geography features into a single feature for easy usage | to categorical

- put the geography location of the samples into a single feature column. 
- france: 1 .. germany: 2 .. spain: 3 .. unknown/not clear: 4

##### why
- better computational usage, as compared to one-hot encoded
- curse of dimensionality
- avoid to many zeros: to reduce memory usage with many zeros.
- data is more than 150K
- what i'm about to do[sparse] might bring more computational overhead as compared one-hot encoded.
- easy to manage less number of features for visualisations and relationship between features

In [43]:
a = np.array(df['France'].values)
b = np.array(df['Germany'].values)
c = np.array(df['Spain'].values)

d = []
for i in range(len(a)):
  if a[i] == 1 and b[i] != 1 and c[i] != 1: d.append(1) # france
  elif b[i] == 1 and a[i] != 1 and c[i] != 1 : d.append(2) # germany
  elif c[i] == 1 and b[i] != 1 and a[i] != 1: d.append(3) # spain
  else: d.append(4) # for unknown
  
# add the new feature column back to the dataset
df['country'] = np.array(d)
# 
# 
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product,country
0,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,1,0,0,0,1,0.0,0.0,0.0,"9.090.909.090.909.090,00",99.0,1
1,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,1,0,0,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0,1
2,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,1,0,0,0,1,0.0,0.0,0.0,0.25,400.0,1
3,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,1,0,0,0,1,1.0,"10.229.405.812.711.500,00","17.606.550.452.171.200,00","58.823.529.411.764.700,00",68.0,1
4,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,0,0,1,0,1,2.0,0.0,0.0,"15.151.515.151.515.100,00",165.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,678.0,33.0,7.0,129735.30,1.0,1,0,86925.09,0,1,0,0,1,0,0.0,"10.119.119.048.366.800,00","14.924.954.348.623.600,00","21.212.121.212.121.200,00",231.0,1
3998,692.0,26.0,2.0,0.00,1.0,1,1,47729.33,0,1,0,0,0,1,1.0,0.0,0.0,"7.692.307.692.307.690,00",52.0,1
3999,699.0,41.0,2.0,134870.79,1.0,1,1,105961.48,0,1,0,0,1,0,1.0,"8.897.071.106.405.830,00","12.728.284.844.643.500,00","4.878.048.780.487.800,00",82.0,1
4000,727.0,33.0,1.0,0.00,2.0,1,1,56767.67,0,0,0,1,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0,3


### Make male and features features into a single feature for easy usage | to categorical

- make the female and male features, into a single sparse feature called [gender]. 
- female: 1 .. male: 0

In [44]:
a = np.array(df['Female'].values)
b = np.array(df['Male'].values)

d = []
for i in range(len(a)):
  if a[i] == 1 and b[i] != 1: d.append(1) # female
  elif b[i] == 1 and a[i] != 1: d.append(0) # male
  else: d.append(3) # for unknown
  
# add the new feature column back to the dataset
df['gender'] = np.array(d)
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain,Female,Male,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product,country,gender
0,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,1,0,0,0,1,0.0,0.0,0.0,"9.090.909.090.909.090,00",99.0,1,0
1,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,1,0,0,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0,1,0
2,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,1,0,0,0,1,0.0,0.0,0.0,0.25,400.0,1,0
3,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,1,0,0,0,1,1.0,"10.229.405.812.711.500,00","17.606.550.452.171.200,00","58.823.529.411.764.700,00",68.0,1,0
4,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,0,0,1,0,1,2.0,0.0,0.0,"15.151.515.151.515.100,00",165.0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,678.0,33.0,7.0,129735.30,1.0,1,0,86925.09,0,1,0,0,1,0,0.0,"10.119.119.048.366.800,00","14.924.954.348.623.600,00","21.212.121.212.121.200,00",231.0,1,1
3998,692.0,26.0,2.0,0.00,1.0,1,1,47729.33,0,1,0,0,0,1,1.0,0.0,0.0,"7.692.307.692.307.690,00",52.0,1,0
3999,699.0,41.0,2.0,134870.79,1.0,1,1,105961.48,0,1,0,0,1,0,1.0,"8.897.071.106.405.830,00","12.728.284.844.643.500,00","4.878.048.780.487.800,00",82.0,1,1
4000,727.0,33.0,1.0,0.00,2.0,1,1,56767.67,0,0,0,1,0,1,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0,3,0


### remove columns turned into sparse fron one-hot encoded
France .. Germany .. Spain .. Female .. male

In [45]:
df.drop(columns=['France', 'Germany', 'Spain', 'Female', 'Male'], inplace=True, axis=1)
print(f"number of columns: {df.columns.shape}")
df

number of columns: (16,)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Mem__no__Products,Cred_Bal_Sal,Bal_sal,Tenure_Age,Age_Tenure_product,country,gender
0,668.0,33.0,3.0,0.00,2.0,1,0,181449.97,0,0.0,0.0,0.0,"9.090.909.090.909.090,00",99.0,1,0
1,627.0,33.0,1.0,0.00,2.0,1,1,49503.50,0,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0,1,0
2,678.0,40.0,10.0,0.00,2.0,1,0,184866.69,0,0.0,0.0,0.0,0.25,400.0,1,0
3,581.0,34.0,2.0,148882.54,1.0,1,1,84560.88,0,1.0,"10.229.405.812.711.500,00","17.606.550.452.171.200,00","58.823.529.411.764.700,00",68.0,1,0
4,716.0,33.0,5.0,0.00,2.0,1,1,15068.83,0,2.0,0.0,0.0,"15.151.515.151.515.100,00",165.0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3997,678.0,33.0,7.0,129735.30,1.0,1,0,86925.09,0,0.0,"10.119.119.048.366.800,00","14.924.954.348.623.600,00","21.212.121.212.121.200,00",231.0,1,1
3998,692.0,26.0,2.0,0.00,1.0,1,1,47729.33,0,1.0,0.0,0.0,"7.692.307.692.307.690,00",52.0,1,0
3999,699.0,41.0,2.0,134870.79,1.0,1,1,105961.48,0,1.0,"8.897.071.106.405.830,00","12.728.284.844.643.500,00","4.878.048.780.487.800,00",82.0,1,1
4000,727.0,33.0,1.0,0.00,2.0,1,1,56767.67,0,2.0,0.0,0.0,"30.303.030.303.030.300,00",33.0,3,0


# Data Visualisation | Features Relationship / Correlation

# Regresion Analysis

# Reflection of the data

# Reflection of the data

# What i learn and whati can do better for next